# Sueño

## Problema Machine Learning

El objetivo del siguiente trabajo es predecir la calidad de sueño de una persona basándonos en datos como duración del sueño, niveles de estrés, nivel de actividad física durante el día (ya sea caminando o haciendo deporte), etc.

El target (calidad de sueño) consiste en un conjunto de valores discretos multiclase (marcados con valores que van del 1 al 5).

Las métricas principales de clasificación son: F1-score, precisión, recall o accuracy.

La métrica secundaria es la matriz de confusión para ver qué clases se confunden más.

**Impacto del proyecto**
- Si el modelo predice una mala calidad del sueño, el sector hospitalario podrá establecer medidas personalizadas acorde a los problemas de las personas.
- Esto podría mejorar la satisfacción del usuario, reduciendo problemas de salud relacionados con el sueño y mejorando la productividad en su día a día.
- Tener una un sueño reparador puede evitar accidentes de tráfico, lo que puede ayudar a salvar vidas.

Los **requisitos mínimos** de rendimiento serán:
- El modelo debe alcanzar al menos un 80% de precisión en la exactitud de los datos (si es de 90% o más sería muy bueno) siempre y cuando las clases estén balanceadas.
- Los valores de F1-score deben tener un mínimo de 0.60 para ser aceptables, si tiene 0.70 serán buenos y si son mayores a 0.80 será excelente.
- La matriz de confusión será buena si no comete errores graves en las clases importantes.

## Recolección y Exploración de Datos

1. Obtener el dataset completo
2. Realizar análisis exploratorio inicial:
   - Dimensiones del dataset
   - Tipos de datos
   - Estadísticas descriptivas básicas
   - Distribución de variables
   - Correlaciones
3. Verificar la calidad de los datos:
   - Identificar valores faltantes
   - Detectar inconsistencias
   - Verificar el balance de clases (para clasificación)
   - Identificar posibles errores en los datos

In [1]:
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore")

In [4]:
df=pd.read_csv('../data/raw/Sleep.csv')

In [5]:
df.head()

,Gender,Age,Sleep duration,Sleep quality,Stress level,Blood pressure,Heart rate,Daily steps,Physical activity,Height,...,Smoking,Medical issue,Ongoing medication,Smart device before bed,Average screen time,Blue-light filter,Discomfort Eye-strain,Redness in eye,Itchiness/Irritation in eye,Dry Eye Disease
0,F,24,9.5,2,1,137/89,67,3000,31,161,...,N,Y,Y,N,8.7,N,Y,Y,N,Y
1,M,39,9.6,2,3,108/64,60,12000,74,164,...,N,Y,Y,N,9.6,Y,N,N,Y,Y
2,F,45,5.4,1,5,134/81,95,12000,93,179,...,N,N,N,Y,4.0,N,Y,N,N,N
3,F,45,5.4,4,5,110/90,78,19000,32,160,...,N,Y,N,N,7.6,N,Y,N,Y,N
4,F,42,5.7,3,2,99/67,72,4000,173,179,...,N,Y,N,N,3.5,N,Y,Y,N,Y


## Preprocesamiento de Datos Inicial

## Feature Engineering

## Preparación de Datos

## Selección de Modelos Base

## Optimización de Modelos Seleccionados

## Evaluación y Análisis

## Divide en train y test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('Sleep quality', axis=1),
                                                    df['Sleep quality'],
                                                    test_size=0.2,
                                                    random_state=42)

## Target

## Comprensión de variables

## Feat. Red. Preliminar

## Análisis Univariante

## Análisis Bivariante

## Eliminación de Features

## Duplicados

## Missings

## Anomalías y Errores

## Outliers

## Feature Engineering

## Feature Reduction

## Escoger Métrica del Modelo

## Decidir sobre los Modelos

## Elegir Hiperparámetros

## Definición de Pipelines y Probamos

## Resultados